In [473]:
import pandas as pd

In [631]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day, CustomBusinessMonthEnd
import datetime
import numpy as np


### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [1]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


Считываем календарь. Возьмём только месяцы и переведём их в числовой формат

In [971]:
df1 = pd.read_csv('./data/data-20191112T1252-structure-20191112T1247.csv', 
                  names=[i for i in range(18)],
                  usecols=[i for i in range(13)],
                  index_col=0, header=0)
df1.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*"
2000,"1,2,3,4,6*,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,2,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,7,11,12,18,19,25,26","2,3,9,10,11*,12,16,17,23,24,30,31"


Сейчас каждое значение датафрейма имеет тип строки, переведём его в список

In [972]:
df2 = df1.applymap(lambda x: str(x).split(','))
df2.head(2)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
1999,"[1, 2, 3, 4, 6*, 7, 9, 10, 16, 17, 23, 24, 30,...","[6, 7, 13, 14, 20, 21, 27, 28]","[6, 7, 8, 13, 14, 20, 21, 27, 28]","[3, 4, 10, 11, 17, 18, 24, 25, 30*]","[1, 2, 3, 4, 8, 9, 10, 15, 16, 22, 23, 29, 30]","[5, 6, 11*, 12, 13, 14, 19, 20, 26, 27]","[3, 4, 10, 11, 17, 18, 24, 25, 31]","[1, 7, 8, 14, 15, 21, 22, 28, 29]","[4, 5, 11, 12, 18, 19, 25, 26]","[2, 3, 9, 10, 16, 17, 23, 24, 30, 31]","[6, 7, 8, 13, 14, 20, 21, 27, 28]","[4, 5, 11, 12, 13, 18, 19, 25, 26, 31*]"
2000,"[1, 2, 3, 4, 6*, 7, 8, 9, 15, 16, 22, 23, 29, 30]","[5, 6, 12, 13, 19, 20, 26, 27]","[4, 5, 7*, 8, 11, 12, 18, 19, 25, 26]","[1, 2, 8, 9, 15, 16, 22, 23, 29, 30]","[1, 2, 6, 7, 8*, 9, 13, 14, 20, 21, 27, 28]","[3, 4, 10, 11, 12, 17, 18, 24, 25]","[1, 2, 8, 9, 15, 16, 22, 23, 29, 30]","[5, 6, 12, 13, 19, 20, 26, 27]","[2, 3, 9, 10, 16, 17, 23, 24, 30]","[1, 7, 8, 14, 15, 21, 22, 28, 29]","[4, 5, 7, 11, 12, 18, 19, 25, 26]","[2, 3, 9, 10, 11*, 12, 16, 17, 23, 24, 30, 31]"


Создадим список выходных и праздников

In [973]:
russian_rules = [Holiday(name='Выходной или праздник', year=year, month=month, day= int(day.partition('+')[0]))
for year in df2.index for month in df2.columns for day in df2.loc[year][month] if day.find('*') == -1]

Создаём список предновогодних выходных (он пригодится в календаре)

In [974]:
holiday_before_new_year = [pd.to_datetime(datetime.datetime(year, 12, 31))
                           for year in df2.index if df2.loc[year][12][-1].find('*') == -1]
print("Список предновогодних выходных")
holiday_before_new_year

Список предновогодних выходных


[Timestamp('2000-12-31 00:00:00'),
 Timestamp('2001-12-31 00:00:00'),
 Timestamp('2005-12-31 00:00:00'),
 Timestamp('2006-12-31 00:00:00'),
 Timestamp('2007-12-31 00:00:00'),
 Timestamp('2011-12-31 00:00:00'),
 Timestamp('2012-12-31 00:00:00'),
 Timestamp('2014-12-31 00:00:00'),
 Timestamp('2016-12-31 00:00:00'),
 Timestamp('2017-12-31 00:00:00'),
 Timestamp('2018-12-31 00:00:00'),
 Timestamp('2022-12-31 00:00:00'),
 Timestamp('2023-12-31 00:00:00')]

Создадим свой календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`

In [975]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = russian_rules
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())

Создадим пустой one-hot DataFrame, который будем заполнять. В нём сейчас месяцы - это столбцы 1-12, а дни недели - столбцы 13-19. Это нужно для удобства заполнения, потом верну нормальные названия столбцам 

In [976]:
data_range_ = pd.date_range(start='1999-01-01', end='2025-12-31', freq='D')
months_ = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
week_days_ = ['пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс'] 
columns_ = ['выходной', 'праздник', 'предпраздничный рабочий день', 'последний день месяца', 'последний рабочий день месяца', 'Предновогодний выходной день'] + list(range(1, 13))+list(range(13, 20))
zeros_ = np.zeros((len(data_range_), len(columns_)), dtype=int)
calendar_df = pd.DataFrame(zeros_, columns=columns_, index=data_range_)
calendar_df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
1999-01-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Заполняем последний день месяца, дни недели и месяцы. Меняем названия столбцов месяцев и дней недели на нормальные

In [977]:
calendar_df.loc[calendar_df.index.is_month_end == True, 'последний день месяца'] = 1
for i in range(1, 13):
    calendar_df.loc[calendar_df.index.month == i, i] = 1
calendar_df.rename(columns={i+1: months_[i] for i in range(12)}, inplace=True)
for i in range(7):
    calendar_df.loc[calendar_df.index.dayofweek == i, i+13] = 1
calendar_df.rename(columns={i+13: week_days_[i] for i in range(7)}, inplace=True)

In [978]:
calendar_df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-01-01,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-02,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-03,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999-01-04,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-01-05,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2025-12-28,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


Заполняем список предновогодних выходных

In [979]:
calendar_df.loc[holiday_before_new_year, 'Предновогодний выходной день'] = 1

In [980]:
calendar_df[calendar_df['Предновогодний выходной день'] == 1]

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
2000-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2001-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2005-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2006-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2007-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2011-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2012-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2014-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2016-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2017-12-31,0,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


Заполняем список выходных

In [981]:
calendar_df.loc[pd.to_datetime(russian_busday.holidays), ['выходной']] = 1

Заполняем список праздников (так как в календаре не разделяют праздники и выходные, то я просто обозначил все государственные праздники)

In [982]:
# Создаём пустой Series, который будем заполнять датами праздников
holidays = pd.Series(pd.date_range('1999-01-02', periods=0, freq=pd.DateOffset(years=1)))
# Заполняем Series датами новогодних праздников (1-8 января)
for i in range(8):
    a = pd.Series(pd.date_range(start=pd.to_datetime('1999-01-01') + pd.offsets.Day() * i, end='2025-12-31', freq=pd.DateOffset(years=1)))
    holidays = pd.concat([holidays, a])
# Заполняем Series остальными праздниками
holidays = pd.concat([holidays,
                      # 23 февраля
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-02-23'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                      # 8 марта
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-03-08'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                      #  1 мая
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-05-01'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                      #  9 мая
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-05-09'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                      # 12 июня
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-06-12'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                      # 4 ноября
                      pd.Series(pd.date_range(start=pd.to_datetime('1999-11-04'), end='2025-12-31', freq=pd.DateOffset(years=1))),
                     ])
# Заполняем праздники в нашем dataframe
calendar_df.loc[holidays, ['праздник']] = 1

Заполняем список последних рабочих дней месяца

In [983]:
# Создаём календарь последних рабочих дней месяца
RussianBusinessMonthEnd = CustomBusinessMonthEnd(n=1,
                       weekmask='Mon Tue Wed Thu Fri', 
                       holidays=russian_rules)
# Создаём data_range последних рабочих дней месяца
russian_business_month_ends = pd.date_range(
    start=pd.to_datetime('1999-01-01'), end='2025-12-31', freq=RussianBusinessMonthEnd)
# Заполняем их в нашем data_frame
calendar_df.loc[russian_business_month_ends, ['последний рабочий день месяца']] = 1

Заполняем список предпраздничных рабочих дней

In [984]:
# Сделано тупо, но будет приятно, если вы объясните, как лучше
# Создал список рабочих дней 'рабочий день' и копию праздников 'праздник*'. Сместил значения из копии праздников на один день вверх и 
# по условиям что 'рабочий день' == 1 и 'праздник*' == 1 записал 1 в столбец 'предпраздничный рабочий день'
calendar_df.insert(len(calendar_df.columns), 'рабочий день', np.zeros(len(calendar_df), dtype=int))
calendar_df.insert(len(calendar_df.columns), 'праздник*', np.zeros(len(calendar_df), dtype=int))
calendar_df.loc[holidays, ['праздник*']] = 1
calendar_df.loc[(calendar_df['праздник*'] == 0), ['рабочий день']] = 1
calendar_df['праздник*'] = calendar_df['праздник*'].shift(-1)
calendar_df.loc[(calendar_df['праздник*'] == 1) & (calendar_df['рабочий день'] == 1), ['предпраздничный рабочий день']] = 1
calendar_df.drop (columns = ['праздник*', 'рабочий день'], inplace=True)

In [985]:
calendar_df[calendar_df['предпраздничный рабочий день'] == 1]

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-02-22,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-03-07,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1999-04-30,0,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1999-05-08,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-06-11,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-07,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2025-04-30,0,0,1,1,1,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
2025-05-08,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2025-06-11,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


## Получили dataframe

In [996]:
calendar_df

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,Предновогодний выходной день,январь,февраль,март,апрель,...,октябрь,ноябрь,декабрь,пн,вт,ср,чт,пт,сб,вс
1999-01-01,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1999-01-02,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1999-01-03,1,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1999-01-04,1,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1999-01-05,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2025-12-28,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2025-12-29,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2025-12-30,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [986]:
ts = pd.read_csv('./data/sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 index_col='TRADEDATE', parse_dates=True)
ts = ts.CLOSE

Добавляем в календарь пропущенные выходные и праздничные дни

In [987]:
ts_1 = ts.asfreq(pd.offsets.Day()).fillna(method='ffill')

In [988]:
ts_1.head(10)

TRADEDATE
2013-03-25    98.79
2013-03-26    97.20
2013-03-27    96.75
2013-03-28    98.59
2013-03-29    98.76
2013-03-30    98.76
2013-03-31    98.76
2013-04-01    98.07
2013-04-02    98.93
2013-04-03    99.59
Freq: D, Name: CLOSE, dtype: float64

Создаём датафрейм, куда будет записывать

In [989]:
ts_1_df = pd.DataFrame(ts_1)
ts_1_df['weekday'] = ts_1_df.index.weekday
ts_1_df

,CLOSE,weekday
TRADEDATE,,
2013-03-25,98.79,0
2013-03-26,97.20,1
2013-03-27,96.75,2
2013-03-28,98.59,3
2013-03-29,98.76,4
...,...,...
2020-01-06,253.90,0
2020-01-07,253.90,1
2020-01-08,259.15,2


Лаговые переменные рабочих дней с глубинами: 1, 3, 5


In [990]:
for i in range(1, 6, 2):
    ts_1_df['Лаг. перемен. раб. дней с глубиной ' + str(i)]=ts_1.shift(i, freq=russian_busday).values
ts_1_df

C:\Users\rinat\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:837: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


,CLOSE,weekday,Лаг. перемен. раб. дней с глубиной 1,Лаг. перемен. раб. дней с глубиной 3,Лаг. перемен. раб. дней с глубиной 5
TRADEDATE,,,,,
2013-03-25,98.79,0,98.79,98.79,98.79
2013-03-26,97.20,1,97.20,97.20,97.20
2013-03-27,96.75,2,96.75,96.75,96.75
2013-03-28,98.59,3,98.59,98.59,98.59
2013-03-29,98.76,4,98.76,98.76,98.76
...,...,...,...,...,...
2020-01-06,253.90,0,253.90,253.90,253.90
2020-01-07,253.90,1,253.90,253.90,253.90
2020-01-08,259.15,2,259.15,259.15,259.15


Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели

In [991]:
for i in range(1, 6, 2):
    ts_1_df['Rolling ' + str(i) + ' mean'] = ts_1_df.groupby('weekday')['CLOSE'].transform(
        lambda x: x.rolling(window=i).mean().shift(1))
    ts_1_df['Rolling ' + str(i) + ' median'] = ts_1_df.groupby('weekday')['CLOSE'].transform(
        lambda x: x.rolling(window=i).median().shift(1))
    ts_1_df['Rolling ' + str(i) + ' max'] = ts_1_df.groupby('weekday')['CLOSE'].transform(
        lambda x: x.rolling(window=i).max().shift(1))
    ts_1_df['Rolling ' + str(i) + ' min'] = ts_1_df.groupby('weekday')['CLOSE'].transform(
        lambda x: x.rolling(window=i).min().shift(1))
    if i == 1:
        continue
    ts_1_df['Rolling ' + str(i) + ' std'] = ts_1_df.groupby('weekday')['CLOSE'].transform(
        lambda x: x.rolling(window=i).std().shift(1))
ts_1_df       

,CLOSE,weekday,Лаг. перемен. раб. дней с глубиной 1,Лаг. перемен. раб. дней с глубиной 3,Лаг. перемен. раб. дней с глубиной 5,Rolling 1 mean,Rolling 1 median,Rolling 1 max,Rolling 1 min,Rolling 3 mean,Rolling 3 median,Rolling 3 max,Rolling 3 min,Rolling 3 std,Rolling 5 mean,Rolling 5 median,Rolling 5 max,Rolling 5 min,Rolling 5 std
TRADEDATE,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,0,98.79,98.79,98.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-26,97.20,1,97.20,97.20,97.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-27,96.75,2,96.75,96.75,96.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-28,98.59,3,98.59,98.59,98.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-03-29,98.76,4,98.76,98.76,98.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06,253.90,0,253.90,253.90,253.90,254.75,254.75,254.75,254.75,248.333333,248.80,254.75,241.45,6.662269,243.162,241.45,254.75,233.77,8.583244
2020-01-07,253.90,1,253.90,253.90,253.90,254.75,254.75,254.75,254.75,248.340000,248.67,254.75,241.60,6.581208,242.718,241.60,254.75,230.66,9.353575
2020-01-08,259.15,2,259.15,259.15,259.15,254.75,254.75,254.75,254.75,249.663333,248.04,254.75,246.20,4.500226,244.180,246.20,254.75,231.59,8.719412


Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [992]:
# Создаём вспомогательные столбцы с праздничными, рабочими и предпраздничными днями
ts_1_df['праздник'] = calendar_df.loc['2013-03-25':'2020-01-10']['праздник']
ts_1_df['предпраздничный рабочий день'] = calendar_df.loc[min(ts_1_df.index):max(ts_1_df.index)]['предпраздничный рабочий день']
ts_1_df['рабочий день'] = np.zeros(len(ts_1_df), dtype=int)
ts_1_df.loc[(ts_1_df['праздник'] == 0), ['рабочий день']] = 1
ts_1_df

,CLOSE,weekday,Лаг. перемен. раб. дней с глубиной 1,Лаг. перемен. раб. дней с глубиной 3,Лаг. перемен. раб. дней с глубиной 5,Rolling 1 mean,Rolling 1 median,Rolling 1 max,Rolling 1 min,Rolling 3 mean,...,Rolling 3 min,Rolling 3 std,Rolling 5 mean,Rolling 5 median,Rolling 5 max,Rolling 5 min,Rolling 5 std,праздник,предпраздничный рабочий день,рабочий день
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2013-03-25,98.79,0,98.79,98.79,98.79,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
2013-03-26,97.20,1,97.20,97.20,97.20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
2013-03-27,96.75,2,96.75,96.75,96.75,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
2013-03-28,98.59,3,98.59,98.59,98.59,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
2013-03-29,98.76,4,98.76,98.76,98.76,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06,253.90,0,253.90,253.90,253.90,254.75,254.75,254.75,254.75,248.333333,...,241.45,6.662269,243.162,241.45,254.75,233.77,8.583244,1,0,0
2020-01-07,253.90,1,253.90,253.90,253.90,254.75,254.75,254.75,254.75,248.340000,...,241.60,6.581208,242.718,241.60,254.75,230.66,9.353575,1,0,0
2020-01-08,259.15,2,259.15,259.15,259.15,254.75,254.75,254.75,254.75,249.663333,...,246.20,4.500226,244.180,246.20,254.75,231.59,8.719412,1,0,0


In [993]:
# Заполняем данные по дням
for i in ['праздник', 'рабочий день', 'предпраздничный рабочий день']:
    ts_1_df['Expand. ' + str(i) + ' mean'] = ts_1_df[ts_1_df[i] == 1].CLOSE.expanding().mean()
    ts_1_df['Expand. ' + str(i) + ' median'] = ts_1_df[ts_1_df[i] == 1].CLOSE.expanding().median()
    ts_1_df['Expand. ' + str(i) + ' max'] = ts_1_df[ts_1_df[i] == 1].CLOSE.expanding().max()
    ts_1_df['Expand. ' + str(i) + ' min'] = ts_1_df[ts_1_df[i] == 1].CLOSE.expanding().min()
    ts_1_df['Expand. ' + str(i) + ' std'] = ts_1_df[ts_1_df[i] == 1].CLOSE.expanding().std()
print('Праздничный день')
print(ts_1_df.loc['2013-05-01'])


Праздничный день
CLOSE                                           99.300000
weekday                                          2.000000
Лаг. перемен. раб. дней с глубиной 1            99.300000
Лаг. перемен. раб. дней с глубиной 3            99.300000
Лаг. перемен. раб. дней с глубиной 5            99.300000
Rolling 1 mean                                  97.890000
Rolling 1 median                                97.890000
Rolling 1 max                                   97.890000
Rolling 1 min                                   97.890000
Rolling 3 mean                                  97.676667
Rolling 3 median                                97.890000
Rolling 3 max                                  100.560000
Rolling 3 min                                   94.580000
Rolling 3 std                                    2.995702
Rolling 5 mean                                  97.874000
Rolling 5 median                                97.890000
Rolling 5 max                                  100.5600

In [994]:
print('Рабочий день')
print(ts_1_df.loc['2020-01-10'])

Рабочий день
CLOSE                                          258.190000
weekday                                          4.000000
Лаг. перемен. раб. дней с глубиной 1           258.190000
Лаг. перемен. раб. дней с глубиной 3           258.190000
Лаг. перемен. раб. дней с глубиной 5           258.190000
Rolling 1 mean                                 255.000000
Rolling 1 median                               255.000000
Rolling 1 max                                  255.000000
Rolling 1 min                                  255.000000
Rolling 3 mean                                 250.590000
Rolling 3 median                               252.060000
Rolling 3 max                                  255.000000
Rolling 3 min                                  244.710000
Rolling 3 std                                    5.300160
Rolling 5 mean                                 245.624000
Rolling 5 median                               244.710000
Rolling 5 max                                  255.000000
R

In [995]:
print('Предпраздничный день')
print(ts_1_df.loc['2019-02-22'])

Предпраздничный день
CLOSE                                          205.250000
weekday                                          4.000000
Лаг. перемен. раб. дней с глубиной 1           205.250000
Лаг. перемен. раб. дней с глубиной 3           205.250000
Лаг. перемен. раб. дней с глубиной 5           205.250000
Rolling 1 mean                                 208.000000
Rolling 1 median                               208.000000
Rolling 1 max                                  208.000000
Rolling 1 min                                  208.000000
Rolling 3 mean                                 211.573333
Rolling 3 median                               210.430000
Rolling 3 max                                  216.290000
Rolling 3 min                                  208.000000
Rolling 3 std                                    4.261623
Rolling 5 mean                                 211.032000
Rolling 5 median                               210.430000
Rolling 5 max                                  216.